# SEC Form 4 Analysis

In [89]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

In [90]:
form4Data = pd.read_csv("form4.csv")
print(form4Data.columns)
print(form4Data.dtypes)
print(len(form4Data), "rows")
print(form4Data.head(10))

Index(['ISSUER_CIK', 'REPORTER_CIK', 'ACCESSION_NUMBER',
       'NAME_OF_REPORTING_PERSON', 'A_OR_D', 'AMOUNT', 'PRICE',
       'TRANSACTION_DATE', 'TITLE_OF_SECURITY', 'ISSUER_NAME', 'ISSUER_TICKER',
       'IS_DIRECTOR', 'IS_OFFICER', 'IS_TEN_PERCENT_OWNER',
       'IS_OTHER_RELATIONSHIP', 'NEW_AMOUNT_OWNED',
       'DIRECT_OR_INDIRECT_OWNERSHIP'],
      dtype='object')
ISSUER_CIK                        int64
REPORTER_CIK                      int64
ACCESSION_NUMBER                  int64
NAME_OF_REPORTING_PERSON         object
A_OR_D                           object
AMOUNT                          float64
PRICE                           float64
TRANSACTION_DATE                 object
TITLE_OF_SECURITY                object
ISSUER_NAME                      object
ISSUER_TICKER                    object
IS_DIRECTOR                      object
IS_OFFICER                       object
IS_TEN_PERCENT_OWNER             object
IS_OTHER_RELATIONSHIP            object
NEW_AMOUNT_OWNED         

In [91]:
def genSECURL(cik, accessionNumber):
  return "https://sec.gov/Archives/edgar/data/"+cik+"/"+accessionNumber.zfill(18)+"/"+accessionNumber[:-8].zfill(10)+"-"+accessionNumber[-8:-6]+"-"+accessionNumber[-6:]+".txt"

In [92]:
# Get the percentages of each filing added into its own column

def getPercentOfHolding(amount, newAmount):
  if amount > newAmount:
    return (newAmount / amount) * 100
  else:
    return (amount/newAmount) * 100

# form4Data['trade_percent_holding'] = (form4Data['AMOUNT']/form4Data['NEW_AMOUNT_OWNED']) * 100
# form4Data['trade_percent_holding'] = getPercentOfHolding(form4Data['AMOUNT'], form4Data['NEW_AMOUNT_OWNED'])
for row in form4Data.iterrows():
  perc = getPercentOfHolding(row[1]['AMOUNT'], row[1]['NEW_AMOUNT_OWNED'])
  form4Data.at[row[0], 'perc_holding'] = perc
# print(genSECURL(str(form4Data["ISSUER_CIK"].iloc[3]), str(form4Data["ACCESSION_NUMBER"].iloc[3])))

In [105]:
# Visualize purchases and sales percentages
fig = px.histogram(form4Data, x='perc_holding', color='A_OR_D', template='plotly_dark', title='Insider security transactions by percentage of holdings', labels={
  "count": "Number of filings",
  "perc_holding": "Percent holding",
  "A_OR_D": "Acquire/Dispose"
})
fig.show()
fig = px.histogram(form4Data, x='perc_holding', facet_col='A_OR_D', color='A_OR_D', template='plotly_dark', title='Insider security transactions by percentage of holdings', labels={
  "count": "Number of filings",
  "perc_holding": "Percent holding",
  "A_OR_D": "Acquire/Dispose"
})
fig.show()

It's funny how often zipfian distribution pops up in the real world.

From these charts we can see that most trades are purchases.

We can also look at the trades by their values:

In [126]:
# Visualize by values
fig = px.histogram(form4Data, x='AMOUNT', color='A_OR_D', template='plotly_dark', title='Insider security transactions by transaction volume, log scale', labels={
  "count": "Number of filings",
  "AMOUNT": "Transaction Amount",
  "A_OR_D": "Acquire/Dispose"
}, log_y=True)
fig.show()
fig = px.histogram(form4Data, x='AMOUNT', facet_col='A_OR_D', color='A_OR_D', template='plotly_dark', title='Insider security transactions by transaction volume, log scale', labels={
  "count": "Number of filings",
  "AMOUNT": "Transaction Amount",
  "A_OR_D": "Acquire/Dispose"
}, log_y=True)
fig.show()

temp = form4Data[form4Data['AMOUNT'] < 10000000]
fig = px.histogram(temp, x='AMOUNT', color='A_OR_D', template='plotly_dark', title='Insider security transactions by transaction volume, log scale, less than $10M', labels={
  "count": "Number of filings",
  "AMOUNT": "Transaction Amount",
  "A_OR_D": "Acquire/Dispose"
}, log_y=True)
fig.show()

Under $10M, it is evident that the large majority of trades are purchases.

What we can deduce from the 2 sets of charts above is that most trades are below $1M, and are less than 20% of a report's holdings.